In [1]:
import os, sys, random, time
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

from util import read_pickle, group_cycle_truck, manual_stop
from pdpt_route_schedule import gurobi_master_cycle, greedy_fix_MP, MP_to_SP, cpo_sub, greedy_heuristic, time_checker_cluster, calculate_SP_cost, postprocess_pdpt_solution


dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
case_num=1
seed = 0

In [2]:
def find_key(list_dict, value):
    return  [k for item_ in list_dict for k, v in item_.items() if value in v]

In [3]:
def select_subroutes(ins, cargo_route_file, truck_pairs_to_try, seed=0, verbose=0):


    # load data from ins
    truck = ins['truck']
    cargo = ins['cargo']
    # edges = ins['edge']
    # nodes = ins['nodes']
    # constant = ins['constant']
    # node_cargo_size_change = ins['node_cargo_size_change']
    edge_shortest = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    # single_truck_deviation = ins['single_truck_deviation']
    selected_truck = {}
    selected_node = []
    selected_edge = {}
    selected_cargo = {}

    cargo_to_truck_assignment = [{key: list(set([v[0] for v in value]))} for key, value in cargo_route_file.items()]

    # print(cargo_to_truck_assignment)
    
    # for truck_key in truck_keys_shuffle[:2]:
    for truck_key in truck_pairs_to_try:
        if verbose >0:
            print(f'========= START [PDOTW with truck {truck_key}] ========= ')

        truck_value = truck[truck_key]

        selected_truck[truck_key] = truck[truck_key]

        cargo_keys = find_key(cargo_to_truck_assignment, truck_key)
        if len(cargo_keys) > 0:
            for c_key in cargo_keys: 
                selected_cargo[c_key] = cargo[c_key]

        if truck_value[0] not in selected_node:
            selected_node.append(truck_value[0])
        if truck_value[1] not in selected_node:
            selected_node.append(truck_value[1])
    # add undelivered cargos
    for cargo_key, cargo_route in cargo_route_file.items():
        if len(cargo_route) == 0:
                selected_cargo[cargo_key] = cargo[cargo_key]


    for v in selected_cargo.values():
        if v[3] not in selected_node:
            selected_node.append(v[3])
        if v[4] not in selected_node:
            selected_node.append(v[4])


    edges_ = list(set([(i,j) for i in selected_node for j in selected_node]))

    for i,j in edges_:
        selected_edge[(i,j)] = edge_shortest[(i,j)]


    return (selected_cargo, selected_truck, selected_node, selected_edge)

In [8]:
def pdpt_route_schedule_decomposition(path_, ins, subroutes, undelivered_cargos, verbose):
    """
    The main function of PDPT decomposition
    input: ct = count
    
    output:
        infeasible_clusters
        removed_cargo_MP
        removed_cargo_SP

    Three steps to handle removed cargos:
        1. add a removed cargo to a different cluster, resolve the MP-SP
        2. add all removed cargos to an unused truck
        3. LNS
    """    



    truck = ins['truck']
    constant = ins['constant']
    # single_truck_deviation = ins['single_truck_deviation']
        
    selected_cargo, selected_truck, selected_node, selected_edge = subroutes

    if len(undelivered_cargos) > 0:
        print('+++ add undelivered cargos to the sub-problem')
        for cargo_key, cargo_value in undelivered_cargos.items():
            if cargo_key not in selected_cargo.keys():
                selected_cargo[cargo_key] = cargo_value

    # Setting parameters for clustering cargo and truck 
    runtime = 100
    # count_node_cluster = 10
    # conflict_obj_coefficient = 10000
    
      
    ###### Solve the MP+SP for each cluster ######
    
    time_start = time.time()
    
    removed_cargo = []
    # solution_cluster = {}   


    ###### Initialize data structures for the solution of the current cluster ######

    # solution_created_truck = {}

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, created_truck_all = \
    group_cycle_truck(selected_truck) 


    ###### Master Problem Solved by Gurobi MIP ######

    # no variable ordering
    # heuristic = 0.2

    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #s_sol: s^k, if truck k i used or not
    #z_sol: z^{kr}_{ij}, if truck k carries parcel r visit edge (i, j) or not
    #y_sol: y^k_r, if parcel r is carried by truck k
    #u_sol: y^r_i, if parcel r is transfered at node i
    #D_sol: D^k_i, depature time of truck k at node i

    gurobi_log_file = path_ + f'_gurobi/rasd_MP.log'
    

    obj_val_MP, runtime_MP, \
    x_sol, s_sol, z_sol, y_sol, u_sol, D_sol, Db_sol, \
    cost_truck_value, cost_travel_value, cost_transfer_value = \
    gurobi_master_cycle(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node, runtime*1, gurobi_log_file, verbose)

    if obj_val_MP < 0:
        print(f'MP infeasible, stop solving, skip to the next iter of RASD')

        
        return False, {}
    else:
        print(f'MP feasible, move to SP')

    
    ###### The MP should be feasible here ######

    ###### Subproblem Solved by CPO CP ######

    # convert MP solution to CP parameters
    print(f'Convert MP solution to SP parameters')

    truck_MP, truck_nodes, truck_nodes_index, cargo_in, cargo_out, \
    transfer_nodes, cargo_unload, cargo_load = \
    MP_to_SP(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        x_sol, s_sol, z_sol, y_sol, u_sol, D_sol)  
    

    # Evaluating the feasibility of SP
    feasibility_SP, g_sol, h_sol, D_sol = \
    cpo_sub(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        truck_MP, truck_nodes, truck_nodes_index, 
        cargo_in, cargo_out,
        transfer_nodes, cargo_unload, cargo_load,
        runtime)

    if feasibility_SP != 'Feasible':
        # Greedy heuristic for cargo removal
        selected_removed_cargo, \
        truck_MP, truck_nodes, truck_nodes_index, \
        cargo_in, cargo_out, \
        transfer_nodes, cargo_unload, cargo_load = \
        greedy_heuristic(constant, selected_cargo, 
            created_truck_yCycle, created_truck_nCycle, created_truck_all, 
            selected_edge, selected_node,
            truck_MP, truck_nodes, truck_nodes_index,
            cargo_in, cargo_out,
            transfer_nodes, cargo_unload, cargo_load)

        # find removed cargo from SP
        for c in selected_removed_cargo:
            removed_cargo.append([c, -1])
    else:
        print(f'SP Feasible, Check time constraint')
        if time_checker_cluster(constant, selected_cargo, created_truck_all, 
                            selected_edge, truck_MP, truck_nodes, 
                            cargo_unload, cargo_load, g_sol, h_sol, D_sol):
            print('Passing time constraint check!')
        else:
            print('Failed time constraint check, exit!!!!')
            print(f'Failed time constraint check, skip to the next iter of RASD')
            return False, {}
            # sys.exit()
        
    # Calculate SP costs: truck cost + traveling cost + transfer cost
    print(f'Compute cost after solving SP')

    truck_cost, travel_cost, transfer_cost = \
    calculate_SP_cost(constant, selected_cargo, selected_edge, 
        truck_MP, truck_nodes, 
        cargo_in, cargo_out, transfer_nodes)


    

    truck_used, cargo_delivered, cargo_undelivered, \
    trucks_per_cargo, cargo_in_truck, truck_route, cargo_route =  postprocess_pdpt_solution(subroutes, x_sol, s_sol, y_sol, z_sol, u_sol, verbose)
    
    if verbose >0:
        print(f'===== summary of post-processing [{selected_truck.keys()}] route+schedule solution')
        print(f'+++ cargo to truck assignment')
        for key, value in cargo_in_truck.items():
            print(f'      {key}: {value}')
        print(f'+++ truck to cargo assignment')
        for key, value in trucks_per_cargo.items():
            print(f'      {key}: {value}')
        print(f'+++ truck route')
        for key, value in truck_route.items():
            print(f'      {key}: {value}')
        print(f'+++ cargo route')
        for key, value in cargo_route.items():
            print(f'      {key}: {value}')

    res = {'MP': {'x_sol': x_sol,
                  's_sol': s_sol,
                  'z_sol': z_sol,
                  'y_sol': y_sol,
                  'u_sol': u_sol,
                  'D_sol': D_sol,
                  'Db_sol': Db_sol,
                 },
            'SP':{'g_sol': g_sol,
                  'h_sol': h_sol,
                  'D_sol': D_sol,
                 },
            'route':{'removed_cargo':removed_cargo,
                     'used_truck':truck_used,
                     'truck_route': truck_route,
                     'cargo_route': cargo_route,
                     'transfer_nodes': transfer_nodes,
                    },
            'cost':{'truck_cost': truck_cost,
                    'travel_cost': travel_cost,
                    'transfer_cost': transfer_cost,
                    }
            }

    return True, res

In [11]:
max_iter = 1
num_iteration = 1

pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}.pkl')
pdotw_sol_res_filename = os.path.join(dir_, 'out', 'iniSol', f'case{case_num}_iniSol.pkl')

pdpt_ins = read_pickle(pdpt_ins_filename)
truck_list = pdpt_ins['truck']
cargo_list = pdpt_ins['cargo']

pdotw_sol = read_pickle(pdotw_sol_res_filename)
best_cost = sum([c_ for c_ in pdotw_sol['cost'].values()])
cargo_route = pdotw_sol['route']['cargo_route']

pdpt_sol = {}
pdpt_sol['truck_route'] = pdotw_sol['route']['truck_route'].copy()
pdpt_sol['cargo_route'] = pdotw_sol['route']['cargo_route'].copy()
pdpt_sol['transfer_nodes'] = []
pdpt_sol['travel_cost'] = pdotw_sol['cost']['travel_cost']
pdpt_sol['truck_cost'] = pdotw_sol['cost']['truck_cost']
pdpt_sol['transfer_cost'] = 0




# pdpt_sol = convert_pdotw_sol_to_pdpt_sol(pdpt_ins, pdotw_sol, verbose = verbose-2)
# best_cost = sum([c_ for c_ in pdpt_sol['cost'].values()])

undelivered_cargo_key = [key for key, value in cargo_route.items() if len(value)==0]
undelivered_cargos = {}
for cargo_key in undelivered_cargo_key:
    undelivered_cargos[cargo_key] = cargo_list[cargo_key]

print(f'+++ undelivered cargo: {undelivered_cargos}')


# truck_pairs_to_try = unique_pair(list(truck_list.keys()), list(truck_list.keys()))
truck_list_keys = list(truck_list.keys())
truck_pairs_to_try = list(set([(truck_list_keys[i], truck_list_keys[j]) 
                                for i in range(len(truck_list_keys)) 
                                    for j in range(i+1, len(truck_list_keys))
                                        if truck_list_keys[i] in pdotw_sol['route']['used_truck'] and truck_list_keys[j] in pdotw_sol['route']['used_truck']]))

random.Random(seed).shuffle(truck_pairs_to_try)
# print('unique pair of trucks to instantiate PDPT subproblems')
selected_truck_pairs = []
num_success_rasd_iter = 0
iter = 0
while num_success_rasd_iter < num_iteration and iter <=max_iter:
    # subroutes = select_subroutes(pdpt_ins, pdotw_sol['route']['cargo_route'], pdotw_sol['route']['used_truck'], verbose)
    subroutes = select_subroutes(pdpt_ins, pdotw_sol['route']['cargo_route'], truck_pairs_to_try[iter], seed=0, verbose=0)
    print(f'++++++ instantitate PDPT subproblem on truck {list(subroutes[1].keys())}')
    feasibility_flag, res = pdpt_route_schedule_decomposition(dir_+f'/out/impSol/case{case_num}', pdpt_ins, subroutes, undelivered_cargos, verbose = 1)

    if feasibility_flag == True:
        manual_stop()
        # update_pdpt_sol(pdpt_sol, res)

        cost = sum([c_ for c_ in res['cost'].values()])

        if cost < best_cost:
            print('find better_solution in RASD')
            pdpt_sol = res
            num_success_rasd_iter += 1
    iter += 1
    selected_truck_pairs.append(list(subroutes[1].keys()))

+++ undelivered cargo: {'C178': (81, 4, 285, 'N7', 'N19')}
++++++ instantitate PDPT subproblem on truck ['T1', 'T16']
selected_truck {'T1': ('N7', 'N30', 780, 10000), 'T16': ('N11', 'N17', 780, 10000)}
/home/tan/Documents/GitHub/pdpt_2022//out/impSol/case1_gurobi/rasd_MP.log
Set parameter TimeLimit to value 100
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022//out/impSol/case1_gurobi/rasd_MP.log"
Set parameter Heuristics to value 0.2

*********************** MP Feasible **********************

The Gurobi obj value is 115369
The Gurobi runtime is 81.446896

cost_truck_value: 60000.0
cost_travel_value: 53650.0
u_sol is 1 with: N21 C28
u_sol is 1 with: N21 C40
u_sol is 1 with: N21 C43
u_sol is 1 with: N21 C50
u_sol is 1 with: N21 C53
u_sol is 1 with: N21 C95
u_sol is 1 with: N21 C161
u_sol is 1 with: N21 C195
u_sol is 1 with: N21 C198
u_sol is 1 with: N21 C234
u_sol is 1 with: N21 C239
u_sol is 1 with: N21 C252
cost_transfer_value: 1719.0
MP feasible, move to SP
Conver

In [ ]:
def update_pdpt_sol(pdpt_ins, pdpt_sol, res):
    ''' 
        PDPT_suprob = {'MP': {'x_sol': x_sol,
                  's_sol': s_sol,
                  'z_sol': z_sol,
                  'y_sol': y_sol,
                  'u_sol': u_sol,
                  'D_sol': D_sol,
                  'Db_sol': Db_sol,
                 },
            'SP':{'g_sol': g_sol,
                  'h_sol': h_sol,
                  'D_sol': D_sol,
                 },
            'route':{'truck_route': truck_route,
                     'cargo_route': cargo_route,
                    },
            'cost':{'truck_cost': truck_cost,
                    'travel_cost': travel_cost,
                    'transfer_cost': transfer_cost,
                    }
            }

        PDOTW = {'MIP': {'x_sol': x_sol_total,
                   'y_sol': y_sol_total,
                   'S_sol': S_sol_total,
                   'D_sol': D_sol_total,
                   'A_sol': A_sol_total,
                   'Sb_sol': Sb_sol_total,
                   'Db_sol': Db_sol_total,
                   'Ab_sol': Ab_sol_total,
                   'runtime': runtime_pdotw,
                  },
           'route': {'truck_yCycle':list(created_truck_yCycle_total.keys()),
                     'used_truck': truck_used_total,
                     'truck_route': truck_route,
                     'cargo_route': cargo_route,
                    },
           'cost': {'truck_cost' : truck_cost,
                    'travel_cost' : travel_cost,
                    },
          }
    '''
    edge_shortest = pdpt_ins['edge_shortest']
    constant = pdpt_ins['constant']
    cargo_list= pdpt_ins['cargo']

    for truck_key, route in res['route']['truck_route'].items():
        pdpt_sol['truck_route'][truck_key] = route
    
    for truck_key, route in res['route']['cargo_route'].items():
        pdpt_sol['cargo_route'][truck_key] = route

    travel_cost = 0
    truck_cost = 0
    for truck_key, route in pdpt_sol['truck_route']:
        for r_idx in range(len(route)-1):
            travel_cost += edge_shortest[(route[r_idx], route[r_idx+1])] * constant['truck_running_cost']
        if len(route) > 1:
            truck_cost += constant['truck_fixed_cost']

    transfer_cost = 0
    for pair in res['transfer_nodes']:
        pdpt_sol.append(pair)
        node_key, cargo_key = pair
        transfer_cost += int(cargo_list[cargo_key][0] * constant['cargo_reloading_cost'])*2

    print(travel_cost, truck_cost, transfer_cost)

    
        
